# Model 2


Stochastic optimization model used to solve the two-stage problem

In [62]:
using JuMP,HiGHS,DataFrames
using CSV
using DataFrames

### Data Imports

In [63]:
dateiStart = "24_buses_"

# Import data about buses
df_buses = CSV.read(string("data/",dateiStart,"paperdata.csv"), DataFrame,types=Dict("C" => String,"I" => String, "C_cur" => String, "Lambda" => String))
#Import demand data
df_demands = CSV.read(string("data/",dateiStart,"demand.csv"), DataFrame)
# Import the realized wind energy
df_realizedEnergy = CSV.read(string("data/",dateiStart,"realizedEnergy.csv"), DataFrame)
# Import the remaining data
df_other = CSV.read(string("data/",dateiStart,"other_data.csv"), DataFrame)
# Import the realized wind energy
df_realizedEnergy_Scenarios = CSV.read(string("data/15_zones_scenarios.csv"), DataFrame)

Row,Node,Q,Omega,Energy
,Int64,Int64,Int64,String
1,1,1,1,0.1424043634282774;0.144210969386013;0.11988087544620207;0.11671758859344804;0.11960952246048036;0.12482732392742532;0.12715439461913772;0.11961921410390236;0.13055074038928308;0.11931038251089561;0.12722993431488186;0.16117609798741356;0.18277840333637954;0.198588648153477;0.20999632645455624;0.23693158274963935;0.23402132344241613;0.2168685678190485;0.2156496868123431;0.21487491527510108;0.19692054015303043;0.17507165835148314;0.15774903513640248;0.1536693664338863
2,1,1,2,0.24925152559364394;0.23660657293347845;0.2547726078975398;0.2652790149419735;0.2666329641104493;0.2647527310067237;0.28023064024832584;0.29483431221458467;0.30195889560570166;0.31900485217471775;0.3110440256075149;0.3046755155103205;0.2973436978306434;0.2731402298296725;0.2514808375322323;0.22248500814580463;0.18812457313677608;0.17922818173641133;0.14537280849930967;0.12966009374807386;0.1228492341775822;0.11352103053779045;0.11219749225114242;0.11694216466549477
3,1,1,3,0.38113445680697255;0.39934607673258554;0.4095107449387022;0.4199402307948202;0.4635363363600875;0.4838429672761221;0.45815356128816886;0.45962077955999875;0.4608940710033462;0.46237447610888494;0.4735992856532162;0.47453501395669934;0.491932889023617;0.5217230384668552;0.5377119579466558;0.5354619885980622;0.5443787862519629;0.5432984670882294;0.5129083261134971;0.5315965861615052;0.5214812424082307;0.5057010037234905;0.49970110245781685;0.4519300793932842
4,1,1,4,0.3562393871699561;0.34706232693889427;0.3532945402036468;0.3687214774982786;0.35007255108896757;0.3502445038004581;0.3484569469061921;0.33878830001140076;0.3438864381025607;0.35772195470643675;0.35589277676492775;0.357784382542768;0.3588229668138723;0.36820877099069554;0.34497861590284434;0.3374650414691527;0.3201062193718641;0.3235007130748609;0.3337909840791105;0.301732519199888;0.2703964802160043;0.24991415635967834;0.2372936232219796;0.23129461265265638
5,1,1,5,0.3177469258584971;0.3008752289768007;0.29834936539598067;0.29615703918518377;0.28857681157250786;0.2819413650761805;0.266231092998373;0.24709347854298014;0.21962456636810757;0.19789840059008557;0.16359869712043917;0.14625633643043243;0.13622391087882296;0.12193326350196675;0.11342517260144637;0.10470287732420024;0.10833499827307795;0.09532711402325825;0.09732845217421128;0.09208974391807369;0.09512147179098973;0.11708501191761549;0.13257660000960908;0.13745296104775367
6,1,1,6,0.15435208076029977;0.12113098001343195;0.09788173718030112;0.0804328875933887;0.07333247529883019;0.08343308031208146;0.07022760509455989;0.07094668164071445;0.08041706269205363;0.07590600684310625;0.08122946010385991;0.09874598762959325;0.10682768353886044;0.09671283194925716;0.07510856005091932;0.07300849647141684;0.07514930402562282;0.07134691740407961;0.08622084885543177;0.0868854578153834;0.09390762986567582;0.10197668061054715;0.0932664114660432;0.09884809643701631
7,1,1,7,0.21828681813942943;0.21515993483004814;0.20767110010063328;0.1993867294992019;0.18353497909808816;0.1734698614029331;0.17292600677572303;0.17881719196379514;0.19093561011171012;0.19228161887711007;0.18288245115593735;0.19570253710992702;0.19275894564364662;0.18051490786014765;0.18200145725032488;0.19751892864922307;0.20982077916088915;0.22134182252453458;0.23930782615262358;0.2590937895924542;0.27382928991344613;0.30202949779754606;0.3277103985144704;0.32992782961622075
8,1,1,8,0.25862492418151733;0.25975650254816807;0.24953751035224667;0.23068437678046524;0.2031330752775938;0.1766320865840305;0.13413771756306855;0.10186319657323725;0.07619941396756216;0.05355012904949481;0.0553493166326664;0.061423000891508685;0.061145140086509925;0.058198814145218325;0.06644235726996184;0.08028433959449277;0.09008971066994341;0.10236930231901496;0.10750230455808549;0.09510652694450542;0.10787319529756394;0.11167002300560692;0.12410473262649876;0.13724691845094805
9,1,1,9,0.17621019591993808;0.16972795408279373;0.16349928548822829;0.1764269595148843;0.21129128126853516;0.23

In [64]:
"""
Transforms indices into range for lists of objects (such as generation units)
Input of 0 or empty cells is translated to no objects (i.e. empty array)
"""
function string_to_range(s)
    # Check if s is missing or a string that represents an empty cell
    if s == "missing" || s == "0"
        return Int[]  # Return an empty array of Int type
    else
        str = string(s)  # Convert to string if not already
        nums = split(str, ";")  # Split the string by comma
        nums = parse.(Int, nums)  # Parse each part to an integer
        return minimum(nums):maximum(nums)  # Return the range
    end
end

"""
Transforms indices into array of values (such as generation costs per unit for each generation unit)
Input of missing cells is translated to no values (i.e. empty array)
Input of 0 is translated to 0 since it can be a coefficient
"""
function string_to_array(s)
    # Check if s is missing or a string that represents an empty cell
    #if s == "0" return 0  # Return an empty array of Float64 type

    if s == "missing"
        return Float64[]
    else
        str = string(s)  # Convert to string if not already
        nums = split(str, ";")  # Split the string by semicolon
        nums = parse.(Float64, nums)  # Parse each part to a float
        return nums  # Return the array of floats
    end
end
"""
Transforms a range and an array into one dictionary
"""
function array_to_dict(keys::AbstractVector{T}, values::AbstractVector{S}) where {T,S}
    # Überprüfen, ob die Länge der Arrays gleich ist
    if length(keys) != length(values)
        throw(ArgumentError("Die Länge der Arrays muss gleich sein"))
    end

    # Initialisierung des leeren Dictionary
    result_dict = Dict{T,S}()

    # Hinzufügen von Schlüssel-Wert-Paaren zum Dictionary
    for i in 1:length(keys)
        result_dict[keys[i]] = values[i]
    end

    return result_dict
end


array_to_dict

In [65]:
# Create an empty dictionary to hold all nodes
data = Dict()

# Iterate over the range of nodes
for singleNode in eachrow(df_buses)
    # Create a dictionary for the current node with all other variables
    data_node = Dict(
        :I => string_to_array(string.(singleNode.I)),
        :Q => string_to_array(string.(singleNode.Q)),
        :J => string_to_array(string.(singleNode.J)),
        :Λ => string_to_array(string.(singleNode.Lambda)),
        :C_save => string_to_array(string(singleNode.C)) ,
        :C_RU_save => string_to_array(string(singleNode.C_RU)),
        :C_RD_save => string_to_array(string(singleNode.C_RD)),
        :C_U_save => string_to_array(string(singleNode.C_U)),
        :C_D_save => string_to_array(string(singleNode.C_D)),
        :V_LOL_save => string_to_array(string(singleNode.V_LOL)),
        :C_cur_save => string_to_array(string(singleNode.C_cur)),
        :P_max_save => string_to_array(string(singleNode.P_max)),
        :b_save => string_to_array(string(singleNode.b)),
        :LC_Max_save => string_to_array(string(singleNode.LC_Max)),
        :L_save => Dict(),
        :C => Dict(),
        :C_RU => Dict(),
        :C_RD => Dict(),
        :C_U => Dict(),
        :C_D => Dict(),
        :P_max => Dict(),
        :C_cur => Dict(),
        :b => Dict(),
        :LC_Max => Dict(),
        :V_LOL => Dict(),
        :Ω => 0,
        :T => 1:1,
    )    
    # Assign the dictionary to the current node key
    data[singleNode.Node] = data_node
end

In [66]:
data[3][:Q]

1-element Vector{Float64}:
 1.0

In [67]:
for singleNode in keys(data)
    # Create a dictionary for the current node with all other variables
    data[singleNode][:C] = array_to_dict(data[singleNode][:I], data[singleNode][:C_save])
    data[singleNode][:C_RU] = array_to_dict(data[singleNode][:I], data[singleNode][:C_RU_save])
    data[singleNode][:C_RD] = array_to_dict(data[singleNode][:I], data[singleNode][:C_RD_save])
    data[singleNode][:C_U] = array_to_dict(data[singleNode][:I], data[singleNode][:C_U_save])
    data[singleNode][:C_D] = array_to_dict(data[singleNode][:I], data[singleNode][:C_D_save])
    data[singleNode][:P_max] = array_to_dict(data[singleNode][:I], data[singleNode][:P_max_save])
    data[singleNode][:C_cur] = array_to_dict(data[singleNode][:Q], data[singleNode][:C_cur_save])
    data[singleNode][:b] = array_to_dict(data[singleNode][:Λ], data[singleNode][:b_save])
    data[singleNode][:LC_Max] = array_to_dict(data[singleNode][:Λ], data[singleNode][:LC_Max_save])
    data[singleNode][:V_LOL] = array_to_dict(data[singleNode][:J], data[singleNode][:V_LOL_save])
end

In [68]:
#Create dictionary for alle demands dependent on node and load -> array with different times
knownLoads = []
demands = Dict()
defaultPeriods = 1
for singleNode in eachrow(df_demands)
    data_node = Dict(
        :L => string_to_array(string(singleNode.Demands)),
    )    
    if  singleNode.Node ∉ knownLoads
        demands = Dict()
    end
    push!(knownLoads, singleNode.Node)

    demands[singleNode.Loads] = data_node
    data[singleNode.Node][:L] = demands
    
    data[singleNode.Node][:T] = 1 : singleNode.Periods
    if defaultPeriods < singleNode.Periods
        defaultPeriods = singleNode.Periods
    end
end

#Fill periods of Nodes without demands -> Set Default Value
for singleNode in keys(data)
    #if data[singleNode][:T] === nothing
        data[singleNode][:T] = minimum(1):maximum(defaultPeriods)
    #end
end


In [69]:
#create dictionary with all scenarios
scenarios = Dict()
for singleNode in eachrow(df_other)
    data_node = Dict(
        :π => singleNode.Pi,
    )
    scenarios[singleNode.Omega] = data_node
end

"""
#create dictionary with all realized Energy from wind turbine dependent on node, wind turbine, scenario -> array with different times
windenergy = Dict()
data_Omega = Dict()
for singleNode in eachrow(df_realizedEnergy)
    data_node = Dict(
        :W_realized => string_to_array(string(singleNode.energy)),
    )
    if ! haskey(windenergy, singleNode.Q)
        data_Omega = Dict()
    end
    data_Omega[singleNode.Omega] = data_node
    windenergy[singleNode.Q] = data_Omega
    data[singleNode.Node][:Windenergy] = windenergy
end
"""

#Float muss in Integer umgewandelt werden und Windturbinen speichern
nodesWithWindTurbines = []#Float muss in Integer umgewandelt werden
for value in keys(data)
    data[value][:Λ] = round.(Int, data[value][:Λ])
    if length(data[value][:Q]) > 0
        push!(nodesWithWindTurbines, value)
    end
end

#create dictionary with all realized Energy from wind turbine dependent on node, wind turbine, scenario -> array with different times
windenergy = Dict()
data_Omega = Dict()
for singleNode in eachrow(df_realizedEnergy_Scenarios)
    data_node = Dict(
        :W_realized => string_to_array(string(singleNode.Energy)),
    )
    if ! haskey(windenergy, singleNode.Q)
        data_Omega = Dict()
    end
    data_Omega[singleNode.Omega] = data_node
    windenergy[singleNode.Q] = data_Omega
    data[nodesWithWindTurbines[singleNode.Q]][:Windenergy] = windenergy
end

#Windenergy must be converted into max values in MW
for n in keys(data)
    for q in data[n][:Q]
        for ω in keys(scenarios)
            for t in data[n][:T]
                data[n][:Windenergy][q][ω][:W_realized][t] = data[n][:Windenergy][q][ω][:W_realized][t] * 300
            end
        end
    end
end

### Functions for Variables and Constraints

In [70]:
# Initializes variables
function init_variables(model::JuMP.Model)
    P=@variable(model, P[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #energy generated
    R_U=@variable(model, R_U[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #committed upward reserve capacity of generator i
    R_D=@variable(model, R_D[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #committed downward reserve capacity of generator i
    r_U=@variable(model, r_U[n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]] >=0) #up regulation of generator i in case 𝜔
    r_D=@variable(model, r_D[n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]] >=0) #down regulation of generator i in case 𝜔
    L_Shed=@variable(model, L_Shed[n in keys(data), j in data[n][:J], ω in keys(scenarios), t in data[n][:T]] >=0) #loss of load at load demand j in case 𝜔
    W_spill=@variable(model, W_spill[n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]] >=0) #curtailment of turbine q in case 𝜔
    W_s=@variable(model, W_s[n in keys(data), q in data[n][:Q], t in data[n][:T]] >=0) #scheduled wind power generation at turbine q
    𝛿=@variable(model, 𝛿[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]] >=0) #voltage angle
    𝛿_scenario=@variable(model, 𝛿_scenario[n in keys(data), ℓ in data[n][:Λ], ω in keys(scenarios),t in data[n][:T]] >=0) #voltage angle in case 𝜔
    PF=@variable(model, PF[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]]) #power flow
    PF_scenario=@variable(model, PF_scenario[n in keys(data), ℓ in data[n][:Λ], ω in keys(scenarios), t in data[n][:T]]) #power flow in case 𝜔
    
    
    vars= Dict(
        :P => P,
        :R_U => R_U,
        :R_D => R_D,
        :r_U => r_U,
        :r_D => r_D,
        :L_Shed => L_Shed,
        :W_spill => W_spill,
        :W_s => W_s,
        :𝛿 => 𝛿,
        :𝛿_scenario => 𝛿_scenario,
        :PF => PF,
        :PF_scenario => PF_scenario,
    )    
    return vars
end;

In [71]:
# Initializes constraints
function init_constraints(model::JuMP.Model, data::Dict, vars::Dict)
    @constraints(model, begin 
    c1[n in keys(data),i in data[n][:I], t in data[n][:T]], vars[:P][n, i, t]+ vars[:R_U][n, i, t]<= data[n][:P_max][i]
    
    c2[n in keys(data),i in data[n][:I] ,t in data[n][:T]], vars[:P][n, i, t]- vars[:R_D][n, i, t]>= 0
    c3[n in keys(data),i in data[n][:I], t in data[n][:T], ω in keys(scenarios)], vars[:r_U][n, i, ω, t] <= vars[:R_U][n,i,t]
    c4[n in keys(data),i in data[n][:I], t in data[n][:T], ω in keys(scenarios)], vars[:r_D][n, i, ω, t] <= vars[:R_D][n,i,t]

    c5[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF][n, ℓ, t] <= data[n][:LC_Max][ℓ]
    c5_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF_scenario][n, ℓ, ω, t] <= data[n][:LC_Max][ℓ]
    
    c6[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:𝛿][n, ℓ, t] - vars[:𝛿][ℓ, n, t]) * 1 / data[n][:b][ℓ]  == vars[:PF][n, ℓ, t]
    c6_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:𝛿_scenario][n, ℓ, ω, t] - vars[:𝛿_scenario][ℓ, n, ω, t]) * 1 / data[n][:b][ℓ] == vars[:PF_scenario][n, ℓ, ω, t]
       
    #steht im Lehrbuch, aber macht keinen Sinn -> muss auskommentiert bleiben
    #c8[n in keys(data), t in data[n][:T]], vars[:𝛿][n, data[n][:Λ][1], t] == 0
    #c81[ω in keys(scenarios), n in keys(data), t in data[n][:T]], vars[:𝛿_scenario][ω, n, data[n][:Λ][1], t] == 0
    #c11, vars[:𝛿][1,2,1]==0  ### hard coded
    #c12[ω in keys(scenarios)], vars[:𝛿_scenario][1,2,ω,1]==0 ### hard coded
    
    c9[ω in keys(scenarios), n in keys(data),t in data[n][:T], q in data[n][:Q]], vars[:W_spill][n,q,ω,t] <= data[n][:Windenergy][q][ω][:W_realized][t]
    c10[ω in keys(scenarios), n in keys(data),t in data[n][:T], j in data[n][:J]], vars[:L_Shed][n,j, ω, t] <= data[n][:L][j][:L][t]

    Power_balance_day_ahead[n in keys(data), t in data[n][:T]], 
    sum(vars[:P][n,i,t] for i in data[n][:I]) + sum(vars[:W_s][n,q,t] for q in data[n][:Q]) - sum(data[n][:L][j][:L][t] for j in data[n][:J]) - sum(vars[:PF][n, ℓ, t] for ℓ in data[n][:Λ]) == 0

    Power_balance_at_stage[ω in keys(scenarios), n in keys(data), t in data[n][:T]], 
    sum(vars[:r_U][n,i, ω, t] for i in data[n][:I]) -
    sum(vars[:r_D][n,i, ω, t] for i in data[n][:I]) +
    sum(vars[:L_Shed][n,j, ω, t] for j in data[n][:J]) +
    sum(data[n][:Windenergy][q][ω][:W_realized][t] -vars[:W_s][n,q,t] - vars[:W_spill][n,q,ω,t] for q in data[n][:Q]) -
    sum(vars[:PF_scenario][n, ℓ, ω, t] for ℓ in data[n][:Λ]) +
    sum(vars[:PF][n, ℓ, t] for ℓ in data[n][:Λ]) == 0
    end)
end;

In [72]:
# Initializes objective function
function init_obj_function(model, data, vars)
    @objective(model, Min, sum(vars[:P][n,i,t]* data[n][:C][i] + vars[:R_U][n,i,t] * data[n][:C_RU][i] + vars[:R_D][n,i,t] * data[n][:C_RD][i] for n in keys(data), i in data[n][:I], t in data[n][:T]) +
    sum(scenarios[ω][:π] * (sum(data[n][:C_U][i] * vars[:r_U][n,i, ω ,t] - data[n][:C_D][i] * vars[:r_D][n,i, ω, t] for i in data[n][:I]) +
                sum(data[n][:C_cur][q] * vars[:W_spill][n,q, ω, t] for q in data[n][:Q]) +
                sum(data[n][:V_LOL][j] * vars[:L_Shed][n, j, ω, t] for j in data[n][:J])) for n in keys(data), ω in keys(scenarios), t in data[n][:T])
    )
end;

### Model Creation

In [73]:
# Create and print model
model=Model(HiGHS.Optimizer)
vars=init_variables(model)
init_constraints(model, data, vars)
init_obj_function(model, data, vars)
#print(model)

13.32 P[1,1.0,1] + 15 R_U[1,1.0,1] + 14 R_D[1,1.0,1] + 13.32 P[1,1.0,2] + 15 R_U[1,1.0,2] + 14 R_D[1,1.0,2] + 13.32 P[1,1.0,3] + 15 R_U[1,1.0,3] + 14 R_D[1,1.0,3] + 13.32 P[1,1.0,4] + 15 R_U[1,1.0,4] + 14 R_D[1,1.0,4] + 13.32 P[1,1.0,5] + 15 R_U[1,1.0,5] + 14 R_D[1,1.0,5] + 13.32 P[1,1.0,6] + 15 R_U[1,1.0,6] + 14 R_D[1,1.0,6] + 13.32 P[1,1.0,7] + 15 R_U[1,1.0,7] + 14 R_D[1,1.0,7] + 13.32 P[1,1.0,8] + 15 R_U[1,1.0,8] + 14 R_D[1,1.0,8] + 13.32 P[1,1.0,9] + 15 R_U[1,1.0,9] + 14 R_D[1,1.0,9] + 13.32 P[1,1.0,10] + 15 R_U[1,1.0,10] + 14 R_D[1,1.0,10] + [[...9204 terms omitted...]] + W_spill[21,9.0,3,19] + W_spill[21,9.0,3,20] + W_spill[21,9.0,3,21] + W_spill[21,9.0,3,22] + W_spill[21,9.0,3,23] + W_spill[21,9.0,3,24] + W_spill[21,9.0,1,1] + W_spill[21,9.0,1,2] + W_spill[21,9.0,1,3] + W_spill[21,9.0,1,4] + W_spill[21,9.0,1,5] + W_spill[21,9.0,1,6] + W_spill[21,9.0,1,7] + W_spill[21,9.0,1,8] + W_spill[21,9.0,1,9] + W_spill[21,9.0,1,10] + W_spill[21,9.0,1,11] + W_spill[21,9.0,1,12] + W_spill[21,

In [74]:
optimize!(model)

In [75]:
value.(vars[:W_s][13,6,1])

155.1226099999999

In [76]:
data[13][:Windenergy][6][10][:W_realized][1]

78.76790242825751

In [78]:
# Print the values of the variables
println("Variable values:")
println("P:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("P[$n, $i, $t]: ", JuMP.value.(vars[:P][n, i, t]))
end
println("\nR_U:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("R_U[$n, $i, $t]: ", value(vars[:R_U][n,i, t]))
end
println("\nR_D:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("R_D[$n, $i, $t]: ", value(vars[:R_D][n,i, t]))end

println("\nL_Shed:")
for n in keys(data), j in data[n][:J], ω in keys(scenarios), t in data[n][:T]
    println("L_Shed[$n, $j, $ω, $t]: ", value(vars[:L_Shed][n,j, ω, t]))
end

println("\nW_spill:")
for n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]
    println("W_spill[$n, $q, $ω, $t]: ", value(vars[:W_spill][n,q, ω, t]))
end

println("\nW_s:")
for n in keys(data), q in data[n][:Q], t in data[n][:T]
    println("W_s[$n, $q, $t]: ", value(vars[:W_s][n,q, t]))
end

println("\nPF[n, ℓ, t]:")
for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]
    println("PF[$n, $ℓ, $t]: ", value(vars[:PF][n, ℓ, t]))
end

println("\nPF_scenario[ω, n, ℓ, t]:")
for ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]
    println("PF_scenario[$ω, $n, $ℓ, $t]: ", value(vars[:PF_scenario][ω, n, ℓ, t]))
end

println("\nr_U[n,i, ω, t]:")
for n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]
    println("r_U[$n, $i, $ω, $t]: ", value(vars[:r_U][n,i, ω, t]))
end

println("\nr_D[n,i, ω, t]:")
for n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]
    println("r_D[$n, $i, $ω, $t]: ", value(vars[:r_D][n,i, ω, t]))
end

println("\nData values:")

println("\nW_realized:")
for n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]
    println("W_realized[$n, $q, $ω, $t]: ", value(data[n][:Windenergy][q][ω][:W_realized][t]))
end

println("\nLoad:")
for n in keys(data), j in data[n][:J], t in data[n][:T]
    println("L[$n, $j, $t]:", value(data[n][:L][j][:L][t]))
end

UndefVarError: UndefVarError: `n` not defined